## Discussion #8

Files needed = `gameshow.csv`, `auto_data.dta`, `monthly_loc_rides.csv`

OH: Mondays and Wednesdays 9:15-10:15am in 6473 Sewell Social Sciences

Email: minnie.cui@wisc.edu

**Reminder:** Coding practice #3 solutions are posted and your Project Proposals are due the Friday after spring break!

#### This week we saw some *very* important and useful topics:
- Multi-indexing
- Reshaping
- Merging
- Intro: transformations

### Multi-indexing

We've seen one-dimensional indexing before with `.set_index()` and `.reset_index()`. You can think of this as a number in a list. When we use multi-indexing, we're creating "sub-lists." We can slice multi-indexed data in different ways. Using the example below, we could look at just Wisconsin or we could look at Madison within Wisconsin. It's a more powerful way to slice data quickly.

We'll use `.set_index()` in pretty much the same way, but we'll also have some other methods such as `.xs()`. This gives us a cross-section of the data at whichever index we'd like. We can also automatically set a multi-index with `index_col` in `read_csv`. There can be multi-indexing for columns, too.

1. Wisconsin
    1. Madison
        1. East
        2. West
    2. Milwaukee
2. Minnesota
    1. Minneapolis
    2. Ely
3. Michigan
    1. Detroit
    2. Grand Rapids
    3. Kalamazoo

In [1]:
# Import libraries
import pandas as pd

# Generate data
soccer = {'team' : ['Man City', 'Man City', 'Man City', 'Man City', 'Chelsea', 'Chelsea'], 
          'player' : ['Walker', 'Stones', 'Foden', 'Jesus', 'Cahill', 'Pedro'],
          'pos' : ['D', 'D', 'M', 'F', 'D', 'F'],
          'goals' : [1, 0, 0, 1, 0, 3],
          'assists': [0,0,0,0,0,0]
         }

# Set index
prem = pd.DataFrame(soccer)
prem.set_index(['team', 'pos'], inplace=True)
prem

player  goals  assists
team     pos                        
Man City D    Walker      1        0
         D    Stones      0        0
         M     Foden      0        0
         F     Jesus      1        0
Chelsea  D    Cahill      0        0
         F     Pedro      3        0

In [6]:
# Slice data
prem.sort_index(axis=0, inplace=True)
#prem.loc[('Man City', 'D'),:] 
#prem.xs('Man City', level='team', drop_level=False) 
#prem.xs('D', level='pos', drop_level=False)
prem.loc[[('Chelsea', 'D'), ('Man City', 'F'), ('Man City', 'D')]] # .xs can't subset multiple

player  goals  assists
team     pos                        
Chelsea  D    Cahill      0        0
Man City F     Jesus      1        0
         D    Walker      1        0
         D    Stones      0        0

### Reshaping

A major part of data cleaning is reshaping the data into a workable form for the task at hand. Sometimes we'll need long data and sometimes we'll need wide data. *Long* data doesn't have many variables but it has a ton of rows. Often times it'll have a variable that describes which variable is recorded as a value. *Wide* data doesn't have many rows but it has a ton of variables.

Indexing is super important for reshaping. It'll let us move between long and wide data easily. Python calls long data *stacked* and wide data *unstacked*. We'll use the functions `.stack()` and `.unstack()` very often.

In [43]:
# Gameshow data
game = pd.read_csv('gameshow.csv')
game

,series,contestant,episode,var,value
0,1,Frank Skinner,1,score,19.0
1,1,Frank Skinner,1,win,1.0
2,1,Frank Skinner,2,score,9.0
3,1,Frank Skinner,2,win,0.0
4,1,Frank Skinner,3,score,15.0
...,...,...,...,...,...
1395,14,Sarah Millican,8,win,0.0
1396,14,Sarah Millican,9,score,18.0
1397,14,Sarah Millican,9,win,0.0
1398,14,Sarah Millican,10,score,15.0


In [44]:
game = game.set_index(['series', 'contestant', 'episode', 'var'])
game = game.sort_index()
game

value
series contestant     episode var         
1      Frank Skinner  1       score   19.0
                              win      1.0
                      2       score    9.0
                              win      0.0
                      3       score   15.0
...                                    ...
14     Sarah Millican 8       win      0.0
                      9       score   18.0
                              win      0.0
                      10      score   15.0
                              win      0.0

[1400 rows x 1 columns]

In [8]:
# Unstack variable
game_us = game.unstack('var') 
game_us.head(10)

value     
var                          score  win
series contestant    episode           
1      Frank Skinner 1        19.0  1.0
                     2         9.0  0.0
                     3        15.0  0.0
                     4        14.0  0.0
                     5        20.0  1.0
                     6        16.0  0.0
                     7         NaN  NaN
                     8         NaN  NaN
                     9         NaN  NaN
                     10        NaN  NaN

In [12]:
# Unstack to wide data
game_us.dropna(inplace=True)
game_us2 = game.unstack(['episode', 'var'])
game_us2
game_us2.xs('score', level='var', axis=1, drop_level=False)

value                                            \
episode                      1     2     3     4     5     6     7     8    
var                       score score score score score score score score   
series contestant                                                           
1      Frank Skinner       19.0   9.0  15.0  14.0  20.0  16.0   NaN   NaN   
       Josh Widdicombe     13.0  16.0  22.0  16.0  18.0   9.0   NaN   NaN   
       Roisin Conaty        7.0  21.0   9.0   9.0  14.0   8.0   NaN   NaN   
       Romesh Ranganathan  19.0  14.0  22.0  10.0  16.0  12.0   NaN   NaN   
       Tim Key             17.0  18.0  16.0   9.0  10.0  18.0   NaN   NaN   
...                         ...   ...   ...   ...   ...   ...   ...   ...   
14     Dara Ó Briain       17.0  30.0  17.0  19.0  19.0  10.0  21.0   9.0   
       Fern Brady          17.0  14.0  14.0  15.0  19.0   6.0  16.0  12.0   
       John Kearns         11.0  10.0  20.0  12.0  23.0  13.0  18.0   6.0   
       Munya Chawawa       14.0  21.0  11.0   9.0  20.0  14.0   8.0  21.0   
       Sarah Millican      18.0  16.0  17.0  22.0  17.0  19.0  15.0  17.0   

                                       
episode                      9     10  
var                       score score  
series contestant                      
1      Frank Skinner        NaN   NaN  
       Josh Widdicombe      NaN   NaN  
       Roisin Conaty        NaN   NaN  
       Romesh Ranganathan   NaN   NaN  
       Tim Key              NaN   NaN  
...                         ...   ...  
14     Dara Ó Briain       22.0  20.0  
       Fern Brady          23.0   8.0  
       John Kearns         19.0  12.0  
       Munya Chawawa       16.0  16.0  
       Sarah Millican      18.0  15.0  

[70 rows x 10 columns]

In [14]:
game_us = game.unstack('var')     #unstack var
game_us

value     
var                           score  win
series contestant     episode           
1      Frank Skinner  1        19.0  1.0
                      2         9.0  0.0
                      3        15.0  0.0
                      4        14.0  0.0
                      5        20.0  1.0
...                             ...  ...
14     Sarah Millican 6        19.0  1.0
                      7        15.0  0.0
                      8        17.0  0.0
                      9        18.0  0.0
                      10       15.0  0.0

[700 rows x 2 columns]

In [32]:
# Stack data
game_s = game_us.stack(dropna=True)
game_s                                             #back to og setup
game_s = game_s.reset_index()
game_s
game_s.rename(columns={0 :'value'}, inplace=True)
game_s

,series,contestant,episode,var,value
0,1,Frank Skinner,1,score,19.0
1,1,Frank Skinner,1,win,1.0
2,1,Frank Skinner,2,score,9.0
3,1,Frank Skinner,2,win,0.0
4,1,Frank Skinner,3,score,15.0
...,...,...,...,...,...
1215,14,Sarah Millican,8,win,0.0
1216,14,Sarah Millican,9,score,18.0
1217,14,Sarah Millican,9,win,0.0
1218,14,Sarah Millican,10,score,15.0


### Merging

We'll often work with multiple datasets for a single project and need to bring everything together. Merging is useful to do this. The goal is to line up the observations in each file correctly. We can left join, right join, inner join, and outer join following the image below.

In words, left (right) join will keep everything in your "left" ("right") dataset and inner join will only keep rows that have keys in both. You need to specify a key to join on, i.e., how you want to match your two datasets, otherwise you'll create all possible combinations of rows.

<img src="merge.png" alt= “” width=400 height=400>

In [33]:
# Import pandas
import pandas as pd

# Create data
df1 = pd.DataFrame(
    {'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Larry', 'Sue'], # 
     'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Bird', 'Legal']} # 
)
df2 = pd.DataFrame(
    {'name': ['Bob', 'Jake', 'Lisa', 'Sue', 'Hannah', 'Sue'], # 
     'salary': [70000, 80000, 120000, 90000, 1000000, 300000]} # 
)

# Merge
pd.merge(df1, df2, left_on="employee", right_on="name", how='inner')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000
4,Sue,HR,Sue,300000
5,Sue,Legal,Sue,90000
6,Sue,Legal,Sue,300000


### Transformations

We saw a little bit of transformation in the last lecture. The `.map()`, `.applymap()`, and `.apply()` methods are especially useful here. Instead of using a loop to apply a function to each row, we can simply put the function name in any of these methods and it will automatically get applied to the data.

There are some key differences between these methods. `.map()` can only be applied to a Series (a single column in a DataFrame). The differences between `.applymap()` and `.apply()` are more subtle. `.apply()` is applied along an *axis* of a DataFrame (columns or rows) whereas `.applymap()` is applied element-wise (not along an axis but to all values).

In [39]:
cars = pd.read_stata('auto_data.dta')
cars.head(3)

,CODE,ORIG,FIRM_ID,FIRM,BRAND,MODEL,YEAR,PRICE,QUANTITY,HP,LENGTH,WIDTH,SIZE,WEIGHT,FUEL,MPG,FUELPRICE,SEGMENT
0,10102,1,4.0,Fiat,Alfa Romeo,ALFA 164,1995,27.581303,179.0,0.048032,179.133865,69.291336,1.241243,3039.647461,0,40.554237,0.70969,4
1,10103,1,4.0,Fiat,Alfa Romeo,ALFA 145,1995,20.202333,4934.0,0.041019,161.023621,67.322838,1.084057,2511.013184,0,38.559769,0.70969,2
2,10104,1,4.0,Fiat,Alfa Romeo,ALFA 155,1995,23.651426,1017.0,0.048181,174.803146,67.039368,1.171869,2671.585938,0,35.106655,0.70969,3


In [36]:
# Total sales amount
def get_sales(x):
    return 1000 * x['PRICE'] * x['QUANTITY']

# Scale price
def scale_price(p):
    return 1000 * p

cars = pd.read_stata('auto_data.dta')
cars['FUEL'] = cars['FUEL'].replace({0 : 'gasoline', 1 : 'diesel'})               
cars['SALES'] = cars.apply(get_sales, axis=1)               #compute sales
cars['PRICE'] = cars['PRICE'].map(scale_price)
cars.head()

,CODE,ORIG,FIRM_ID,FIRM,BRAND,MODEL,YEAR,PRICE,QUANTITY,HP,LENGTH,WIDTH,SIZE,WEIGHT,FUEL,MPG,FUELPRICE,SEGMENT,SALES
0,10102,1,4.0,Fiat,Alfa Romeo,ALFA 164,1995,27581.302643,179.0,0.048032,179.133865,69.291336,1.241243,3039.647461,gasoline,40.554237,0.70969,4,4.937053e+06
1,10103,1,4.0,Fiat,Alfa Romeo,ALFA 145,1995,20202.333450,4934.0,0.041019,161.023621,67.322838,1.084057,2511.013184,gasoline,38.559769,0.70969,2,9.967831e+07
2,10104,1,4.0,Fiat,Alfa Romeo,ALFA 155,1995,23651.426315,1017.0,0.048181,174.803146,67.039368,1.171869,2671.585938,gasoline,35.106655,0.70969,3,2.405350e+07
3,10105,1,4.0,Fiat,Alfa Romeo,ALFA 146,1995,15584.880829,22799.0,0.039797,167.322830,67.322838,1.126465,2588.105713,gasoline,38.559769,0.70969,2,3.553197e+08
4,10201,1,19.0,Volkswagen,Audi,100,1995,35495.632172,4.0,0.044074,188.582672,69.685036,1.314139,3017.621094,gasoline,33.602085,0.70969,4,1.419825e+05


4937053173.096999

In [41]:
# Let's use monthly_loc_rides.csv from the coding practice.
# Is this data long or wide? (stacked or unstacked)
# Stack or unstack the data by location, i.e., have a column for each variable and location. What should our index be?
# What multi-indexing do we have now? How can you tell?

monthly_loc_rides = pd.read_csv('monthly_loc_rides.csv').dropna()
monthly_loc_rides.set_index(['year', 'month', 'location'], inplace=True)
monthly_loc_rides.unstack(['location', 'month'])

avg_fare                                                         \
location       1.0        2.0        3.0        4.0        5.0        6.0    
month            11         11         11         11         11         11   
year                                                                         
2018      10.645126   9.895857   9.948583  10.123744  10.014318   9.584586   
2019      11.581945  10.942402  10.539731  10.797725  10.664824   9.985258   
2020      11.653433  10.958382  10.785500  10.822828  10.800803  10.507763   
2021      14.751944  13.481349  14.516139  13.863651  14.279545  15.123060   
2022      14.841113  13.281700  14.201999  13.533507  13.973395  14.829588   

                                                      ... num_rides           \
location       7.0        8.0        9.0        10.0  ...      68.0     69.0   
month            11         11         11         11  ...        10       10   
year                                                  ...                      
2018       9.089151   9.689035  11.336610  10.788369  ...       NaN      NaN   
2019       9.406940  10.144774  12.061555  11.315425  ...   34677.0  44861.0   
2020      10.002515  10.366860  12.129343  11.791392  ...   28060.0  36636.0   
2021      14.076702  16.079053  15.181480  13.495093  ...   23047.0  30568.0   
2022      14.065941  16.602537  14.437729  13.184844  ...   31098.0  40775.0   

                                                                         \
location     70.0     71.0     72.0     73.0    74.0     75.0      76.0   
month          10       10       10       10      10       10        10   
year                                                                      
2018          NaN      NaN      NaN      NaN     NaN      NaN       NaN   
2019      24017.0  44023.0  11772.0  21591.0  7015.0  14490.0  408420.0   
2020      16830.0  39175.0   7229.0  18148.0  3874.0  11430.0   96742.0   
2021      16003.0  31233.0   7235.0  15649.0  4170.0  11781.0  235652.0   
2022      20774.0  40629.0   9035.0  20338.0  5409.0  15503.0  325125.0   

                    
location      77.0  
month           10  
year                
2018           NaN  
2019      125550.0  
2020       64431.0  
2021       77117.0  
2022       90797.0  

[5 rows x 2772 columns]

### Have a great Spring Break! :)